In [14]:

import requests
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import sys
from bs4 import BeautifulSoup
import re
import unicodedata

import matplotlib.pyplot as plt
import seaborn as sns

import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

import folium



# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [15]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    colunm_name = ' '.join(row.contents)

    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name

In [16]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "MiScrapperProyecto/1.0 (davidzamudiotab@gmail.com)"
}

# Realizar la petición
response = requests.get(static_url, headers=headers)

# Verificar el código de estado
print(response.status_code) # Debería ser 200 si tiene éxito

if response.status_code == 200:
    # Continuar con tu código de Beautiful Soup o procesamiento
    print("Petición exitosa, puedes hacer el scraping.")
else:
    print(f"Error: {response.status_code}")

200
Petición exitosa, puedes hacer el scraping.


In [17]:
soup = BeautifulSoup(response.content, 'html.parser')

In [18]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = soup.find_all('table')

In [19]:
# Let's print the first table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 

In [20]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table


# Iterate each th element and apply the provided extract_column_from_header() to get a column name

# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names

In [21]:
column_names = []
# Apply find_all() function with `th` element on first_launch_table
th_elements = first_launch_table.find_all('th')

# Iterate each th element and apply the provided extract_column_from_header() to get a column name
for th in th_elements:
    name = extract_column_from_header(th)
    # Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
    if name is not None and len(name) > 0:
        column_names.append(name)

print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [22]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [23]:
extracted_row = 0
#Extract each table
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit() # will return flag = True if the table header is a digit
        else:
            flag=False
        #get table element
        row=rows.find_all('td')
        #if it is number save cells in a dictonary
        if flag: # if the row happens to be a flagged row since it is a digit
            extracted_row += 1 # will add one row
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            launch_dict['Flight No.'].append(flight_number)
            #print(flight_number)
            try:
                datatimelist=date_time(row[0])
                # Date value
                # TODO: Append the date into launch_dict with key `Date`
                date = datatimelist[0].strip(',')
                launch_dict['Date'].append(date)
                #print(date)

                # Time value
                # TODO: Append the time into launch_dict with key `Time`
                time = datatimelist[1]
                launch_dict['Time'].append(time)
                #print(time)
            except IndexError:
                launch_dict['Date'].append(None)
                launch_dict['Time'].append(None)
                print(f"Error processing date/time for row {extracted_row}: {row}")


            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            try:
                bv=booster_version(row[1])
                if not(bv):
                    bv=row[1].a.string
                launch_dict['Version Booster'].append(bv)
                #print(bv)
            except (IndexError, AttributeError):
                launch_dict['Version Booster'].append(None)
                print(f"Error processing booster version for row {extracted_row}: {row}")


            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            try:
                launch_site = row[2].a.string
                launch_dict['Launch site'].append(launch_site)
                #print(launch_site)
            except (IndexError, AttributeError):
                launch_dict['Launch site'].append(None)
                print(f"Error processing launch site for row {extracted_row}: {row}")


            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            try:
                payload = row[3].a.string
                launch_dict['Payload'].append(payload)
                #print(payload)
            except (IndexError, AttributeError):
                launch_dict['Payload'].append(None)
                print(f"Error processing payload for row {extracted_row}: {row}")

            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            try:
                payload_mass = get_mass(row[4])
                launch_dict['Payload mass'].append(payload_mass)
                #print(payload)
            except (IndexError, AttributeError):
                launch_dict['Payload mass'].append(None)
                print(f"Error processing payload mass for row {extracted_row}: {row}")


            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            try:
                orbit = row[5].a.string
                launch_dict['Orbit'].append(orbit)
                #print(orbit)
            except (IndexError, AttributeError):
                launch_dict['Orbit'].append(None)
                print(f"Error processing orbit for row {extracted_row}: {row}")


            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            try:
                customer = row[6].a.string
                launch_dict['Customer'].append(customer)
                #print(customer)
            except (IndexError, AttributeError):
                # Some rows have multiple customers in a plainlist, handle this case
                customer_list = row[6].find_all('li')
                if customer_list:
                    customers = [cust.text.strip() for cust in customer_list]
                    launch_dict['Customer'].append(', '.join(customers))
                else:
                    launch_dict['Customer'].append(None)
                    print(f"Error processing customer for row {extracted_row}: {row}")


            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            try:
                launch_outcome = list(row[7].strings)[0]
                launch_dict['Launch outcome'].append(launch_outcome)
                #print(launch_outcome)
            except (IndexError, AttributeError):
                launch_dict['Launch outcome'].append(None)
                print(f"Error processing launch outcome for row {extracted_row}: {row}")


            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            try:
                booster_landing = landing_status(row[8])
                launch_dict['Booster landing'].append(booster_landing)
                #print(booster_landing)
            except (IndexError, AttributeError):
                launch_dict['Booster landing'].append(None)
                print(f"Error processing booster landing for row {extracted_row}: {row}")


#After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.

Error processing customer for row 106: [<td>24 January 2021<br/>15:00<sup class="reference" id="cite_ref-624"><a href="#cite_note-624"><span class="cite-bracket">[</span>612<span class="cite-bracket">]</span></a></sup>
</td>, <td><a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">F9 B5</a> ♺<br/><a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1058" title="List of Falcon 9 first-stage boosters">B1058.5</a><sup class="reference" id="cite_ref-625"><a href="#cite_note-625"><span class="cite-bracket">[</span>613<span class="cite-bracket">]</span></a></sup>
</td>, <td><a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">CCSFS</a>,<br/><a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">SLC-40</a>
</td>, <td><i><a class="new" href="/w/index.php?title=Transporter-1&amp;action=edit&amp;redlink=1" title="Transporter-1 (page does not exist)">Transporter-1</a></i> (<a href="/wiki/List_of_spacefl

In [24]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [27]:
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.07B0003.18,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.07B0004.18,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.07B0005.18,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.07B0006.18,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.07B0007.18,No attempt\n,1 March 2013,15:10


In [25]:
df.to_csv('spacex_web_scraped.csv', index=False)

In [26]:
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>